# Extracting Spectra

This notebook demonstrates how to grab 1D aperture summed HETDEX spectra for an input of ID, RA and DEC using the `Extract` Class API from `hetdex_api`. This can be done interactively using `get_spectra` from hte `hetdex_tools.get_spec` module. It can also be done in the command line using the quick entry call `hetdex_get_spec` providing you have hetdex_api pip installed.

Examples of what you might like to do with the spectra afterwards is shown later. The output is stored in an astropy table of spectra. For every HETDEX observation where spectra is found, a spectra is given. It is up to the user to combine the spectra afterwards.

### IMPORTANT NOTE OF CAUTION WITH RUNNING ON TACC!!!

Because this script involves opening the Fibers class object which contains all fiber spectra from a 3 dither observation, you will be pulling in a lot of memory for each shot that is open. **NEVER** run this script from a login node on TACC. A login node is a node you access when you ssh in. 

You need to request a compute node instead by either 

(1) using the idev command : `idev -t 04:00:00` 

(2) using a jupyter notebook 

(3) or by submitting the job into the slurm job scheduler (this is probably only needed if you are doing more than 1000 extractions per shot on more than 1000 shots)

### Import all necessary python packages. 
These are mainly for working within the notebook. The command line tool already has the necessary preamble built in.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table, join

from hetdex_tools.get_spec import get_spectra

## Getting all spectra at a specified RA/DEC. This will search through all shots in HDR2

If a shotid is not specified the program will search for any shot within HDR2 that overlaps within an 11 arcmin radius of the input coordinates. Because of the non-contiguous VIRUS footprint, there is no guarantee the aperture defined by the input ra/dec/rad will contain enough fibers to do a measurement. The aperture radius is 3" by default or can be specified with the --rad argument. 

Open a catalog of IDs, RAs, DECs:

In [ ]:
input_cat = Table.read('api-notebooks/highz_emitters.fits')

In [ ]:
input_cat[0:5]

`get_spectra()` requires an astropy coordinates object list as an input.

In [ ]:
input_coords = SkyCoord(ra=input_cat['RA'], dec=input_cat['DEC'])

In [ ]:
sources = get_spectra(input_coords, ID=input_cat['ID'])

##  get_spectra() options

There are a few options to consider when running get_spectra():

In [ ]:
help(get_spectra)

### Reading in the output - astropy FITS files

In [ ]:
sources[0:5]

Join your input and output table so you can match up any properties you like

In [ ]:
output_table = join(input_cat, sources)

In [ ]:
output_table[0:5]

In [ ]:
sel = output_table['FLUX'] > 10

for row in output_table[sel][4:8]:
    plt.figure()
    wave_obj = row['WAVE']
    wave = row['wavelength']
    spec = row['spec']
    spec_err = row['spec_err']
    plt.errorbar(wave, spec, yerr=spec_err)
    plt.xlim(wave_obj-50, wave_obj+50)
    plt.xlabel('wave')
    plt.ylabel('spec')
    plt.title(row['ID'])


### Examples of running get_spec as a command line job:


You can run these commands from the command line by removing the "!" command but be sure you are on a compute node by calling `idev` first or submitting these as slurm jobs, one task per line.

In [ ]:
!hetdex_get_spec --ra 150.02548 --dec 2.087987 --ID cosmos_LAE --outfile cosmos_LAE

### Speed things up using multiprocessing

You can speed up processes (by up to ~30x) with python multiprocessing if you are working interactively in a notebook or in an idev session (**NEVER FROM A LOGIN NODE**). Use the multiprocessing option with the argument -mp True or --multiprocess True 

In [ ]:
!hetdex_get_spec --multiprocess -ra 150.02548 -dec 2.087987 -id mptest -o mptest

### Save output as individual astropy tables for each ID/shot combination:

If you prefer to save each spectra to a table, you can do this. We don't recommend this for large numbers of data, but understand that sometimes its easy to start with a readable table. The tables will be stored in the files named `spec_[ID]_[shotid].tab`

In [ ]:
!hetdex_get_spec  --multiprocess --single -ra 150.02548 -dec 2.087987 -id cosmos_lae

### Getting all spectra at a specified RA/DEC in a specific OBSERVATION/SHOT

Perhaps you only want to focus on a specific shot. Then you can use the -s argument to put the shotid either as an interger value 'YYYYMMDDOBS'= 20190104008 or as a str '20190104v009'. Note if you don't give an --ID option the default is 'DEX'

This is a command line routine so remove the "!" if you are running in a terminal.

In [ ]:
!hetdex_get_spec  -ra 8.86535 -dec 0.59352  -s 20190104008 -o 20190104008

This is particularly helpful if you plan to submit each shot as a separate task. For this reason, I suggest changing the default --outfile option to -o 20190104008 to create the output fits file 20190104008.fits